### adb调试

In [1]:
import os
import time

os.system("adb shell input keyevent 26")  # 电源打开屏幕
time.sleep(0.5)
os.system("adb shell input swipe 552 1316 552 623")  # 上划
time.sleep(0.5)

In [4]:
#coding=utf-8
import cv2
import numpy as np  
 
img = cv2.imread("./12.png", 0)
 
img = cv2.GaussianBlur(img,(3,3),0) #用高斯平滑处理原图像降噪。
canny = cv2.Canny(img, 50, 150) #最大最小阈值
 
cv2.imshow('Canny', canny)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#coding=utf-8
import cv2
import numpy as np  
 
img = cv2.imread("./12.png", 0)
 
gray_lap = cv2.Laplacian(img,cv2.CV_16S,ksize = 3)
dst = cv2.convertScaleAbs(gray_lap)
 
cv2.imshow('laplacian',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
import cv2
import numpy as np

m1 = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
m2 = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
from matplotlib import pyplot as plt
# 第一步：完成高斯平滑滤波
img = cv2.imread("./12.png", 0)
sobel = cv2.Canny(img, 50, 100)
cv2.namedWindow('5', 0)
cv2.resizeWindow("5", 640, 480)
cv2.imshow("5", sobel)  # 角度值灰度图
img = cv2.GaussianBlur(img, (3, 3), 2)
# 第二步：完成一阶有限差分计算，计算每一点的梯度幅值与方向
img1 = np.zeros(img.shape, dtype="uint8")  # 与原图大小相同
theta = np.zeros(img.shape, dtype="float")  # 方向矩阵原图像大小
img = cv2.copyMakeBorder(img, 1, 1, 1, 1, borderType=cv2.BORDER_REPLICATE)
rows, cols = img.shape
for i in range(1, rows - 1):
    for j in range(1, cols - 1):
        Gy = [np.sum(m2 * img[i - 1:i + 2, j - 1:j + 2])]
        #Gy = (np.dot(np.array([1, 1, 1]), (m2 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
        Gx = [np.sum(m1 * img[i - 1:i + 2, j - 1:j + 2])]
        #Gx = (np.dot(np.array([1, 1, 1]), (m1 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
        if Gx[0] == 0:
            theta[i - 1, j - 1] = 90
            continue
        else:
            temp = ((np.arctan2(Gy[0], Gx[0])) * 180 / np.pi)+90
        if Gx[0] * Gy[0] > 0:
            if Gx[0] > 0:
                # 第一象线
                theta[i - 1, j - 1] = np.abs(temp)
            else:
                # 第三象线
                theta[i - 1, j - 1] = (np.abs(temp) - 180)
        if Gx[0] * Gy[0] < 0:
            if Gx[0] > 0:
                # 第四象线
                theta[i - 1, j - 1] = (-1) * np.abs(temp)
            else:
                # 第二象线
                theta[i - 1, j - 1] = 180 - np.abs(temp)
        
        img1[i - 1, j - 1] = (np.sqrt(Gx[0] ** 2 + Gy[0] ** 2))
for i in range(1, rows - 2):
    for j in range(1, cols - 2):
        if (((theta[i, j] >= -22.5) and (theta[i, j] < 22.5)) or
                ((theta[i, j] <= -157.5) and (theta[i, j] >= -180)) or
                ((theta[i, j] >= 157.5) and (theta[i, j] < 180))):
            theta[i, j] = 0.0
        elif (((theta[i, j] >= 22.5) and (theta[i, j] < 67.5)) or
              ((theta[i, j] <= -112.5) and (theta[i, j] >= -157.5))):
            theta[i, j] = -45.0
        elif (((theta[i, j] >= 67.5) and (theta[i, j] < 112.5)) or
              ((theta[i, j] <= -67.5) and (theta[i, j] >= -112.5))):
            theta[i, j] = 90.0
        elif (((theta[i, j] >= 112.5) and (theta[i, j] < 157.5)) or
              ((theta[i, j] <= -22.5) and (theta[i, j] >= -67.5))):
            theta[i, j] = 45.0
'''
for i in range(1, rows - 1):
    for j in range(1, cols - 1):
        Gy = [np.sum(m2 * img[i - 1:i + 2, j - 1:j + 2])]
        #Gy = (np.dot(np.array([1, 1, 1]), (m2 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
        Gx = [np.sum(m1 * img[i - 1:i + 2, j - 1:j + 2])]
        #Gx = (np.dot(np.array([1, 1, 1]), (m1 * img[i - 1:i + 2, j - 1:j + 2]))).dot(np.array([[1], [1], [1]]))
        if Gx[0] == 0:
            theta[i - 1, j - 1] = 90
            continue
        else:
            temp = (np.arctan2(Gy[0], Gx[0])) * 180 / np.pi)
        if Gx[0] * Gy[0] > 0:
            if Gx[0] > 0:
                # 第一象线
                theta[i - 1, j - 1] = np.abs(temp)
            else:
                # 第三象线
                theta[i - 1, j - 1] = (np.abs(temp) - 180)
        if Gx[0] * Gy[0] < 0:
            if Gx[0] > 0:
                # 第四象线
                theta[i - 1, j - 1] = (-1) * np.abs(temp)
            else:
                # 第二象线
                theta[i - 1, j - 1] = 180 - np.abs(temp)
        
        img1[i - 1, j - 1] = (np.sqrt(Gx[0] ** 2 + Gy[0] ** 2))
for i in range(1, rows - 2):
    for j in range(1, cols - 2):
        if (((theta[i, j] >= -22.5) and (theta[i, j] < 22.5)) or
                ((theta[i, j] <= -157.5) and (theta[i, j] >= -180)) or
                ((theta[i, j] >= 157.5) and (theta[i, j] < 180))):
            theta[i, j] = 90.0
        elif (((theta[i, j] >= 22.5) and (theta[i, j] < 67.5)) or
              ((theta[i, j] <= -112.5) and (theta[i, j] >= -157.5))):
            theta[i, j] = 45.0
        elif (((theta[i, j] >= 67.5) and (theta[i, j] < 112.5)) or
              ((theta[i, j] <= -67.5) and (theta[i, j] >= -112.5))):
            theta[i, j] = 0.0
        elif (((theta[i, j] >= 112.5) and (theta[i, j] < 157.5)) or
              ((theta[i, j] <= -22.5) and (theta[i, j] >= -67.5))):
            theta[i, j] = -45.0

'''
# 第三步：进行 非极大值抑制计算
img2 = np.zeros(img1.shape)  # 非极大值抑制图像矩阵

for i in range(1, img2.shape[0] - 1):
    for j in range(1, img2.shape[1] - 1):
        # 0度j不变
        if (theta[i, j] == 0.0) and (img1[i, j] == np.max([img1[i, j], img1[i + 1, j], img1[i - 1, j]])):
            img2[i, j] = img1[i, j]

        if (theta[i, j] == -45.0) and img1[i, j] == np.max([img1[i, j], img1[i - 1, j - 1], img1[i + 1, j + 1]]):
            img2[i, j] = img1[i, j]

        if (theta[i, j] == 90.0) and img1[i, j] == np.max([img1[i, j], img1[i, j + 1], img1[i, j - 1]]):
            img2[i, j] = img1[i, j]

        if (theta[i, j] == 45.0) and img1[i, j] == np.max([img1[i, j], img1[i - 1, j + 1], img1[i + 1, j - 1]]):
            img2[i, j] = img1[i, j]

# 第四步：双阈值检测和边缘连接
img3 = np.zeros(img2.shape)  # 定义双阈值图像
# TL = 0.4*np.max(img2)
# TH = 0.5*np.max(img2)
TL = 50
TH = 100
# 关键在这两个阈值的选择
for i in range(1, img3.shape[0] - 1):
    for j in range(1, img3.shape[1] - 1):
        if img2[i, j] < TL:
            img3[i, j] = 0
        elif img2[i, j] > TH:
            img3[i, j] = 255
        elif ((img2[i + 1, j] < TH) or (img2[i - 1, j] < TH) or (img2[i, j + 1] < TH) or
              (img2[i, j - 1] < TH) or (img2[i - 1, j - 1] < TH) or (img2[i - 1, j + 1] < TH) or
              (img2[i + 1, j + 1] < TH) or (img2[i + 1, j - 1] < TH)):
            img3[i, j] = 255

cv2.namedWindow('1', 0)
cv2.resizeWindow("1", 640, 480)
cv2.namedWindow('2', 0)
cv2.resizeWindow("2", 640, 480)
cv2.namedWindow('3', 0)
cv2.resizeWindow("3", 640, 480)
cv2.namedWindow('4', 0)
cv2.resizeWindow("4", 640, 480)
cv2.imshow("1", img)  # 原始图像
cv2.imshow("2", img1)  # 梯度幅值图
cv2.imshow("3", img2)  # 非极大值抑制灰度图
cv2.imshow("4", img3)  # 最终效果图
cv2.waitKey(0)



-1